In [3]:
import requests
from bs4 import BeautifulSoup
import time

base_url = "https://www.airbnb.fr/s/Afrique/homes"
params = {
    "tab_id": "home_tab",
    "refinement_paths[]": "/homes",
    "flexible_trip_lengths[]": "one_week",
    "monthly_start_date": "2024-12-01",
    "monthly_length": "3",
    "monthly_end_date": "2025-03-01",
    "price_filter_input_type": "0",
    "channel": "EXPLORE",
    "place_id": "ChIJ1fWMlApsoBARs_CQnslwghA",
    "date_picker_type": "calendar",
    "checkin": "2024-12-20",
    "checkout": "2024-12-23",
    "source": "structured_search_input_header",
    "search_type": "filter_change"
}

# Collecte des URLs d'annonces
response = requests.get(base_url, params=params)
soup = BeautifulSoup(response.content, 'html.parser')

urls = []
for link in soup.find_all('a', href=True):
    if '/rooms/' in link['href']:
        urls.append("https://www.airbnb.fr" + link['href'].split('?')[0])  


In [6]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Installe et configure automatiquement ChromeDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Ouvrir une page web
driver.get("https://www.airbnb.fr/s/Afrique/homes")
print(driver.title)

driver.quit()


Airbnb | Maroc ⋅ Locations de vacances et logements


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Configuration de Selenium avec WebDriver Manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# URL cible
url = "https://www.airbnb.fr/s/Afrique/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2024-12-01&monthly_length=3&monthly_end_date=2025-03-01&price_filter_input_type=0&channel=EXPLORE&place_id=ChIJ1fWMlApsoBARs_CQnslwghA&date_picker_type=calendar&checkin=2024-12-20&checkout=2024-12-23&source=structured_search_input_header&search_type=filter_change"

driver.get(url)
time.sleep(5)  # Temps pour que la page se charge

# Défilement pour charger plus de résultats
scroll_pause_time = 3
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Collecte des URLs des annonces
links = driver.find_elements(By.XPATH, "//a[contains(@href, '/rooms/')]")
urls = list(set([link.get_attribute("href").split('?')[0] for link in links]))

print(f"{len(urls)} URLs collectées.")
driver.quit()


18 URLs collectées.


In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time

# Recharger les URLs collectées
data = []

# Reconfiguration du driver Selenium
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

for url in urls:
    try:
        driver.get(url)
        time.sleep(3)  # Laisser le temps à la page de se charger

        # Extraction des informations
        title = driver.find_element(By.XPATH, "//h1").text
        price = driver.find_element(By.XPATH, "//span[contains(text(), '€')]").text
        rating = driver.find_element(By.XPATH, "//span[@class='_10fy1f8']").text if driver.find_elements(By.XPATH, "//span[@class='_10fy1f8']") else "N/A"
        reviews = driver.find_element(By.XPATH, "//span[contains(@class, '_a7a5sx')]").text if driver.find_elements(By.XPATH, "//span[contains(@class, '_a7a5sx')]") else "N/A"

        data.append({
            "URL": url,
            "Title": title,
            "Price": price,
            "Rating": rating,
            "Reviews": reviews
        })

        time.sleep(2)  # Pause entre chaque requête
    except Exception as e:
        print(f"Erreur lors du traitement de {url}: {e}")

driver.quit()
print(f"Données extraites pour {len(data)} annonces.")


Erreur lors du traitement de https://www.airbnb.fr/rooms/24402395: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[contains(text(), '€')]"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001048ebac4 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x00000001048e4314 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x000000010434c4b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x0000000104390898 cxxbridge1$string$len + 368748
4   chromedriver                        0x00000001043ca0fc cxxbridge1$string$len + 604368
5   chromedriver                        0x00000001043850b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x0000000104385d00 cxxbridge1$string$len + 3248

In [31]:
# Conversion en DataFrame
df = pd.DataFrame(data)

# Sauvegarde dans un fichier CSV
df.to_csv("airbnb_afrique.csv", index=False)
print("airbnb_afrique.csv")


airbnb_afrique.csv


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      16 non-null     object
 1   Title    16 non-null     object
 2   Price    0 non-null      object
 3   Rating   16 non-null     object
 4   Reviews  16 non-null     object
dtypes: object(5)
memory usage: 772.0+ bytes


In [33]:
import matplotlib.pyplot as plt

# Calculer le prix moyen par destination
average_price_by_reviews = df.groupby('Rating')['Reviews'].mean().reset_index()

# Créer le graphique en barres
plt.figure(figsize=(10, 6))
plt.bar(average_price_by_reviews['Rating'], average_price_by_rating['Reviews'], color='skyblue', edgecolor='black')

# Ajouter un titre et des étiquettes
plt.title('Prix Moyen par Avis')
plt.xlabel('Rating')
plt.ylabel('Prix Moyen (€)')
plt.xticks(rotation=45)

# Afficher le graphique
plt.show()


TypeError: agg function failed [how->mean,dtype->object]